# Recommender Systems & Graph Mining


### Setting up the Notebook

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## 1 Recommender Systems (20 Points)

Throughout the task in Part 1 you are free to implement your calculations or using pen and paper, but please make sure that you provide sufficient details in your submitted answer that make it clear how you got your results. The individual tasks specify this a bit more.


### 1.1 Content-based (User-Item Similarities) (7 Points)

The [Spotify Dataset 1921-2020](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks) contains over 175,000 songs with both [audio features](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features) and [track features](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-track). For this task, we look at 6 different songs an individual user $u$ has rated, and limit ourselves to 4 audio features, all ranking from 0 to 1.

The ratings are not part of the dataset but manually added for this task. The range of the ratings is from 1 to 10.

In [ ]:
df = pd.read_csv('data/spotify-sample.csv')

df.head(6)

,acousticness,danceability,energy,liveness,rating
0,0.93,0.32,0.14,0.18,9
1,0.11,0.85,0.82,0.09,2
2,0.75,0.36,0.39,0.12,7
3,0.84,0.50,0.24,0.13,8
4,0.88,0.33,0.18,0.09,8
5,0.11,0.76,0.69,0.09,2


**1.1 a) Calculate the user profile vector $v_u$ based on $u$'s rating history! (5 Points)** Please complete the equation in the markdown cell below; use a precision of 2 decimals.

**Imortant:** Show at least for one element in the profile vector how you calculated the value in detail. You can use an additional code or markdown cell.

**Your anwser:**

Calculation Process:

(1) Normalize user's ratings

mean_rating = (9+2+7+8+8+2)/6 = 6

\[0]: 9-6 = 3 |
\[1]: 2-6 = -4 |
\[2]: 7-6 = 1 |
\[3]: 8-6 = 2 |
\[4]: 8-6 = 2 |
\[5]: 2-6 = -4

(2) Calculate user's profile (weighted features)(precision: 2 decimals)

acousticness: (0.93\*3 + 0.11\*(-4) + 0.75\*1 + 0.84\*2 + 0.88\*2 + 0.11\*(-4))/6 = 1.02

danceability: (0.32\*3 + 0.85\*(-4) + 0.36\*1 + 0.50\*2 + 0.33\*2 + 0.76\*(-4))/6 = -0.58

energy: (0.14\*3 + 0.82\*(-4) + 0.39\*1 + 0.24\*2 + 0.18\*2 + 0.69\*(-4))/6 = -0.73

liveness: (0.18\*3 + 0.09\*(-4) + 0.12\*1 + 0.13\*2 + 0.09\*2 + 0.09\*(-4))/6 = 0.06

* $v_u$ = \[1.02, -0.58, -0.73, 0.06\]

**1.1 b) Calculate all cosine similarities between user $u$ and 2 new songs! (2 Points)** Please complete the table and the statement below; use a precision of 2 decimals for the similarity values.

|      | acousticness | danceability | energy | liveness | cosine similarity            |
| ---  | ---          | ---          | ---    | ---      | ---                          |
| A    | 0.24         | 0.72         | 0.43   | 0.02     | <font color='red'>-0.40</font> |
| B    | 0.79         | 0.32         | 0.12   | 0.09     | <font color='red'>0.45</font> |

The song we should recommend to user $u$ is: <font color="red">B</font>

### 1.2 User-based Collaborative Filtering (7 Points)

Given to you is a simple rating dataset containing 6 users $u_1, u_2, \dots, u_6$, 8 songs $s_1, s_2, \dots, s_8$, and the rating matrix $R$:

$$
R = 
\begin{bmatrix} 
    4 & 0 & 0 & 3 & 5 &  \mathbf{\color{red} ?}  & 1 & 4 \\
    3 & 0 & 0 & 3 & 4 & 1 & 2 & 0 \\
    1 & 0 & 0 & 2 & 0 & 5 & 4 & 2 \\
    4 & 0 & 0 & 4 & 0 & 2 & 1 & 5 \\
    3 & 3 & 0 & 2 & 4 & 0 & 1 & 2 \\
    0 & 1 & 0 & 3 & 4 & 2 & 1 & 4
\end{bmatrix}
$$

In this example, the range of the ratings is from 1 to 5.

Your task is to find the best estimate for rating $R_{u_1,s_6}$ of user $u_1$ for song $s_6$, indicated by the red question mark in rating matrix $R$.

**1.2 a) Calculate all cosine similarities between user $u_1$ and all other user! (5 Points)!** Please complete the list of equations in the markdown below; please use a precision of 2 decimals.

**Imortant:** Show at least for one equation how you calculate the similarity in detail. You can use an additional code or markdown cell.

**Your answer:**

Calculation Process:

(1) Normalize rating vectors

user_mean_ratings = \[(4+3+5+1+4)/5, (3+3+4+1+2)/5, (1+2+5+4+2)/5, (4+4+2+1+5)/5, (3+3+2+4+1+2)/6, (1+3+4+2+1+4)/6 \] = \[3.4, 2.6, 2.8, 3.2, 2.5, 2.5\]

normalize: subtracting the average user rating except for value 0

u1 = \[4-3.4, 0, 0, 3-3.4, 5-3.4, 0, 1-3.4, 4-3.4\] = \[0.6, 0, 0, -0.4, 1.6, 0, -2.4, 0.6\]

u2 = \[3-2.6, 0, 0, 3-2.6, 4-2.6, 1-2.6, 2-2.6, 0\] = \[0.4, 0, 0, 0.4, 1.4, -1.6, -0.6, 0\]

u3 = \[1-2.8, 0, 0, 2-2.8, 0, 5-2.8, 4-2.8, 2-2.8\] = \[-1.8, 0, 0, -0.8, 0, 2.2, 1.2, -0.8\]

u4 = \[4-3.2, 0, 0, 4-3.2, 0, 2-3.2, 1-3.2, 5-3.2\] = \[0.8, 0, 0, 0.8, 0, -1.2, -2.2, 1.8\]

u5 = \[3-2.5, 3-2.5, 0, 2-2.5, 4-2.5, 0, 1-2.5, 2-2.5\] = \[0.5, 0.5, 0, -0.5, 1.5, 0, -1.5, -0.5\]

u6 = \[0, 1-2.5, 0, 3-2.5, 4-2.5, 2-2.5, 1-2.5, 4-2.5\] = \[0, -1.5, 0, 0.5, 1.5, -0.5, -1.5, 1.5\]

(2) Calculate similarity between normalized user vectors: (precision of 2 decimals)

cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)

$sim(u1, u2) = \frac{dot(u1, u2)}{norm(u1)*norm(u2)} = \frac{(0.6*0.4 + 0*0 + 0*0 + (-0.4)*0.4 + 1.6*1.4 + 0*(-1.6) + (-2.4)*(-0.6) + 0.6*0)} {(\sqrt{(0.6*0.6+0*0+0*0+(-0.4)*(-0.4)+1.6*1.6+0*0+(-2.4)*(-2.4)+0.6*0.6)} * \sqrt{(0.4*0.4+0*0+0*0+0.4*0.4+1.4*1.4+(-1.6)*(-1.6)+(-0.6*-0.6)+0*0)})} = 0.54$

Similarly,

$sim(u1, u3) = -0.41$

$sim(u1, u4) = 0.65$

$sim(u1, u5) = 0.87$

$sim(u1, u6) = 0.72$

* sim($u_1$, $u_2$) = 0.54
* sim($u_1$, $u_3$) = -0.41
* sim($u_1$, $u_4$) = 0.65
* sim($u_1$, $u_5$) = 0.87
* sim($u_1$, $u_6$) = 0.72

**1.2 b) Calculate the estimated rating $R_{u_1,s_6}$ (2 Points)!** Consider the 2 most similar users for this calculation. Show how you arrived at this result! You can use an additional code or markdown cell.

Calculation Process:

2 most similar users: u5, u6

sim(u1, u5) = 0.87; sim(u1, u6) = 0.72

R(u5, s6) = 0; R(u6, s6) = 2

$R_{u_1,s_6} = \frac{0.87*0+0.72*2}{0.87+0.72} = 0.91$

* $R_{u_1,s_6}$ = 0.91

### 1.3 Questions

**1.3 a) (3 Points)** Assume you get the two song rating datasets from 1.1 and 1.2 (just for many more users and songs). Can you combine both datasets to make better recommendations, and if so, what are the requirements?

**Your answer:**

Yes. Requirements:

(1) The songs have the same features. For example,  in 1.1 datasets the songs have 4 audio features, all ranking from 0 to 1, songs from 1.2 datasets should also have same features (acousticness,danceability,energy,liveness).

(2) The rating range of users from 1.2 should be the same or can be converted to the rating range from 1.1. For instance, the range of ratings in 1.2 is 1-5 while in 1.1 is 1-10, they should be converted to the same scale (and make sure that conversion would not lose information) before combination.  

(3) The songs from 1.2 should be also in 1.1 datasets, so that their features' values are known.(or we have to filter songs that exist in 1.1, 1.2 movies without features cannot be combined into 1.1)

**1.3 b) (3 Points)** In 1.2, the song $s_3$ is new has not received any rating yet, so $R_{u,s_3} = 0$ for any user $u$. Thus, using basic Collaborative Filtering, $s_3$ will never me recommended and therefore is less likely to get more ratings. For real-world recommender systems, what are a common strategies to mitigate this problem?

**Your answer:**

This is a cold start problem which can commonly happen in Collaborative Filting. In real-world recommenders, to mitigate this problem, below propose some strategies:

(1)Cold-items can be recommended based on content (content-based filtering). In this case, even if it might have no interactions with users as a new item, the recommender system can choose this item (song s3) to recommend based on the feature s3 possesses, so that its features will allow for a recommendation to be made.

This assumes that the new item can be described by its attributes. In this case song, it has some features that are always known when it is added to the catalogue, such as author, year, form, style, rhythmic features, etc. So it can be recommended to users that might have similar tastes to certain features.

(2) Quickly complete its profile before making recommendations. Another strategy is that the recommender can automatically assign some ratings to the new item, based on the ratings of other similar items, and the item similarity can be determined by their features based on content. Machine learning can be applied to this strategy to generate initial ratings.



--------------------------------------------------

## 2 Graph Mining (30 Points)


### Data Preparation

As graph dataset, we use the bus network in Singapore collected from the [LTA DataMall](https://datamall.lta.gov.sg/content/datamall/en/dynamic-data.html); for more details you can check the [API Documentation](https://datamall.lta.gov.sg/content/dam/datamall/datasets/LTA_DataMall_API_User_Guide.pdf). On the following, we load and prepare the data for the task. You can just run through the following code cells to get everything set up


#### Load Bus Stops

In [ ]:
## Load data from file into a dataframe
df_bus_stops = pd.read_csv('data/sg-bus-stops-march2021.csv', dtype=str)

## Create a dictionary that maps a bus stop code to its description for quick access
bus_stop_map = { str(row['BusStopCode']):row['Description']   for idx, row in df_bus_stops.iterrows() }

## Show the 5 first row of the dataframe to see how the data looks like
df_bus_stops.head()

,BusStopCode,Description,Latitude,Longitude,RoadName
0,01012,Hotel Grand Pacific,1.29684825487647,103.85253591654006,Victoria St
1,01013,St. Joseph's Ch,1.29770970610083,103.8532247463225,Victoria St
2,01019,Bras Basah Cplx,1.29698951191332,103.85302201172507,Victoria St
3,01029,Opp Natl Lib,1.2966729849642,103.85441422464267,Nth Bridge Rd
4,01039,Bugis Cube,1.29820784139683,103.85549139837407,Nth Bridge Rd


#### Load Bus Connections

The data of the bus connections contains information about each direct bus connection between 2 bus stops. Each bus stop is identified by its unique bus stop number. The data also contains the service number of the bus and the distance in kilometers.

In [ ]:
## Load data from file into a dataframe
df_bus_network = pd.read_csv('data/sg-bus-network-march2021.csv')

## Show the 5 first row of the dataframe to see how the data looks like
df_bus_network.head()

,svcno,origin,destination,distance
0,10,75009,76059,0.6
1,10,76059,76069,0.5
2,10,76069,96289,1.2
3,10,96289,96109,0.4
4,10,96109,85079,0.6


#### Generate Graph

From the bus connection data, we can no generate the bus network graph where the nodes are bus stops and there is a directed edge between two nodes if there is a direct bus connection between the corresponding bus stops. While the distance between two bus stops would make a meaningful edge weight, we ignore it in this notebook for simplicity and assume an unweighted graph.

**Note:** The code below loops over each row of the dataframe. Since different bus services provide a direct connection between the same 2 bus stops, the data frame naturally contains duplicate (origin, destination) pairs. However, a `nx.DiGraph` supports only single edges between nodes, so the duplicate pairs do not result in multiple edges but get simple overwritten.

In [ ]:
## Create an "empty" directed graph G
G = nx.DiGraph()

for idx, row in df_bus_network.iterrows():
    G.add_edge(row['origin'], row['destination'])

#### Basic Information about the Graph

In [ ]:
print('The graph has {} nodes and {} edges'.format(len(G.nodes), len(G.edges)))

The graph has 5045 nodes and 7439 edges


As the number of edges is of the same magnitude as the number of nodes -- that is, $|E|\in O(|V|)$ -- graph G is considered to be sparse, which is very common for many to most real-world applications.

With the code cell below, we *could* plot the graph, but its size makes it impractical. As a side note, for visualizing large graphs there a better tools available (e.g., [Gephi](https://gephi.org/)).

In [ ]:
#plt.figure()
#plt.axis('off')
#plt.tight_layout()
#nx.draw(G, with_labels=False, font_weight='bold')
#plt.show()

#### Create Adjacency Matrix for Graph G

We later also need the adjacency matrix of the bus network graph. We can simple use [`to_numpy_matrix()`](https://networkx.org/documentation/networkx-1.7/reference/generated/networkx.convert.to_numpy_matrix.html) which returns the graph adjacency matrix as a numpy matrix. As the identifier of nodes are the bus stop codes but the matrix can only be accessed via its row and column indices, we also need a dictionary that maps the indices back to the bus top codes

In [ ]:
## Convert graph of bus connection to adjacency matrix
A_mrt = nx.to_numpy_matrix(G)

## Create a map to maps in matrix index to the bus stop code (which are the node identifiers)
node_map = { idx:node for idx, node in enumerate(G.nodes)  }

## Print an example
print(node_map[0])

75009


This means that the node represented by index 0 represents the bus stop with the code "75009".

### 2.1 Implementing PageRank (14 Points)

I this task, you will implement the basic PageRank algorithm using the Power Iteration methods as introduced in the lecture.

$$
c_{PR} = \alpha M c_{PR} + (1-\alpha)E
$$

where $E = (1/n, 1/n, ..., 1/n)^T$ with $n$ being the number of nodes.

For testing and debugging your implementation, define a very basic graph by means of its adjacency matrix. In fact, this toy graph is the one used in the [original PageRank paper](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) (Page 5) from 1999.

In [ ]:
A_demo = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [1, 0, 0]    
], dtype=np.float)

<ipython-input-11-7907a87df62f>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ], dtype=np.float)


**2.1 a) Implement method `create_transition_matrix()` that converts and adjacency matrix to a transition matrix! (3 Points).** (Hint: You can find an example for this transformation in the lecture slides.)

In [ ]:
def create_transition_matrix(A):
    
    M = None
    
    #########################################################################################
    ### Your code starts here ###############################################################    
    
    #Transpose A
    M = A.T 
    #Normalize columns
    M = M /M.sum(axis = 0)
    M = np.nan_to_num(M) # nan -> 0
    
    ### Your code ends here #################################################################
    #########################################################################################
    
    return M


create_transition_matrix(A_demo)

array([[0. , 0. , 1. ],
       [0.5, 0. , 0. ],
       [0.5, 1. , 0. ]])

**2.1 b) Implement method `pagerank()` to compute the PageRank scores an adjacency matrx A (9 Points).** (Hint: The algorithm in pseudo code can be found in the lecture slides)

In [ ]:
def pagerank(A, alpha=0.85, eps=0.0001, max_iter=1000, verbose=True):
    
    ## Generate transition matrix from adjacency matrix A
    M = create_transition_matrix(A)

    
    E, c = None, None
    
    #########################################################################################
    ### Your code starts here ############################################################### 

    ## Initialize E and c
    N = A.shape[0]
    E = np.ones(N) / float(N)
    c = np.ones(N) / float(N) 
    
    ### Your code ends here #################################################################
    ######################################################################################### 

    # Run the power method: iterate until differences between steps converges
    num_iter = 0
    while True:

        #########################################################################################
        ### Your code starts here ###############################################################  
        num_iter += 1
        c_prev = c
        c = alpha * np.dot(np.asarray(M),c) + (1-alpha) * E
        c = c/sum(c) #normalize vector
        diff = np.sum(np.abs(c-c_prev)) #calculate difference
        if diff <= eps:
            break
        if num_iter >= max_iter:
            verbose = False
            break
            
        ### Your code ends here #################################################################
        #########################################################################################            
            
        # REMOVE AFTER ADDING YOUR CODE!!! (just here to avoid an infinite loop :) )
        #break
        
    if verbose == True:
        print('The power method took {} iterations.'.format(num_iter))
        
    ## Return the results as a dictiory with the nodes as keys and the PageRank score as values
    return { k:score for k, score in enumerate(c.squeeze()) }


Test your implementation on the toy graph.

In [ ]:
pagerank(A_demo, alpha=1.0)

The power method took 25 iterations.


{0: 0.39998372395833337, 1: 0.19999186197916669, 2: 0.4000244140625}

With $\alpha=1.0$ the scores should look as follows -- apart from floating point precision issues:

`{0: 0.4, 1: 0.2, 2: 0.4}`

Again, you can check with the [original PageRank paper](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) (Page 5).

**2.1 c) Run your PageRank implementation on the bus network graph (2 Points)**. Find the top-5 bus stops with respect to their PageRank scores and complete the table below. Use the code cell below to show how you get to the results; use the default value $\alpha = 0.85$. (Hint: You will need `node_map` and `bus_stop_map` to get the description of a bus stop from an index)


| Rank | Score | Bus Stop Description |
| ---  | ---   | ---                  |
| 1    | 0.00160   | Boon Lay Int                  |
| 2    | 0.00103   | W'Lands Temp Int                  |
| 3    | 0.00098   | Bef Joo Koon Int                  |
| 4    | 0.00094   | Bedok Int                  |
| 5    | 0.00086   | Choa Chu Kang Int                  |

(for the scores, please use a precision of 5 decimals)

In [ ]:
#########################################################################################
### Your code starts here ###############################################################
mrt_pr_scores = pagerank(A_mrt)
keys = sorted(mrt_pr_scores, key=mrt_pr_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1)
    print('Score:','{:.5f}'.format(mrt_pr_scores[key]))
    print('Bus Stop: ',bus_stop_map[node_map[key]])

### Your code ends here #################################################################
#########################################################################################  

The power method took 30 iterations.
No. 1
Score: 0.00160
Bus Stop:  Boon Lay Int
No. 2
Score: 0.00103
Bus Stop:  W'Lands Temp Int
No. 3
Score: 0.00098
Bus Stop:  Bef Joo Koon Int
No. 4
Score: 0.00094
Bus Stop:  Bedok Int
No. 5
Score: 0.00086
Bus Stop:  Choa Chu Kang Int


### 2.2 Comparing Centrality Measures (10 Points)

We saw in the lecture that different centrality measures look at different topological features of a graph to quantify the importance of nodes. This task compares different measures, using the following implementations provided by `networkX`:

* [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)
* [nx.centrality.in_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.in_degree_centrality.html#networkx.algorithms.centrality.in_degree_centrality)
* [nx.centrality.out_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.out_degree_centrality.html#networkx.algorithms.centrality.out_degree_centrality)
* [nx.centrality.closeness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality)
* [nx.centrality.betweenness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html)

(**Note:** [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) might give a (slightly) different ranking than your own implementation of PageRank since this implementation is a bit modified. So don't take this as a 1:1 reference to check your PageRank implementation.)

**2.2 a) Run the 5 centrality measures on the bus network graph (5 Points)**. Find the top-5 bus stops with respect to their centrality scores and complete the table below. You only need to add the description of the bus stops, not the scores.

Use the code cell below to show how you get to the results; use the default values for all 5 implementations of the centrality measures. (Hint: You will need `bus_stop_map` to get the description of a bus stop from an index)


| Rank | PageRank | InDegree | OutDegree | Closeness | Betweenness |
| ---  | ---      | ---      | ---       | ---       |  ---        |
| 1    | Boon Lay Int      | Boon Lay Int      | Opp MAS Bldg       | Mapletree Anson       | UIC Bldg         |
| 2    | W'Lands Temp Int      | Mapletree Anson      | Boon Lay Int       | Aft Capital Twr       | Aft Capital Twr         |
| 3    | Bef Joo Koon Int      | Dhoby Ghaut Stn      | The Esplanade       | Raffles Pl Stn Exit F       | Dhoby Ghaut Stn         |
| 4    | Bedok Int      | Bedok Int      | Dhoby Ghaut Stn Exit B       | OPP SO SOFITEL S'PORE       | Opp MAS Bldg         |
| 5    | Aft Capital Twr      | W'Lands Temp Int      | Bedok Int       | Opp Suntec City       | Mapletree Anson         |

In [ ]:
%%time

#########################################################################################
### Your code starts here ###############################################################
G = nx.DiGraph(A_mrt)
print('PageRank')
pr_scores = nx.algorithms.link_analysis.pagerank(G)
keys = sorted(pr_scores, key=pr_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1, ', Score:','{:.5f}'.format(pr_scores[key]), ', Bus Stop: ',bus_stop_map[node_map[key]])

print('InDegree')
id_scores = nx.centrality.in_degree_centrality(G)
keys = sorted(id_scores, key=id_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1, ', Score:','{:.5f}'.format(id_scores[key]), ', Bus Stop: ',bus_stop_map[node_map[key]])

print('OutDegree')
od_scores = nx.centrality.out_degree_centrality(G)
keys = sorted(od_scores, key=od_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1, ', Score:','{:.5f}'.format(od_scores[key]), ', Bus Stop: ',bus_stop_map[node_map[key]])

print('Closeness')
cl_scores = nx.centrality.closeness_centrality(G)
keys = sorted(cl_scores, key=cl_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1, ', Score:','{:.5f}'.format(cl_scores[key]), ', Bus Stop: ',bus_stop_map[node_map[key]])

print('Betweenness')
bt_scores = nx.centrality.betweenness_centrality(G)
keys = sorted(bt_scores, key=bt_scores.get, reverse=True)[:5]
for index,key in enumerate(keys):
    print('No.', index+1, ', Score:','{:.5f}'.format(bt_scores[key]), ', Bus Stop: ',bus_stop_map[node_map[key]])
    
### Your code ends here #################################################################
#########################################################################################      

PageRank
No. 1 , Score: 0.00158 , Bus Stop:  Boon Lay Int
No. 2 , Score: 0.00103 , Bus Stop:  W'Lands Temp Int
No. 3 , Score: 0.00100 , Bus Stop:  Bef Joo Koon Int
No. 4 , Score: 0.00094 , Bus Stop:  Bedok Int
No. 5 , Score: 0.00086 , Bus Stop:  Aft Capital Twr
InDegree
No. 1 , Score: 0.00218 , Bus Stop:  Boon Lay Int
No. 2 , Score: 0.00218 , Bus Stop:  Mapletree Anson
No. 3 , Score: 0.00159 , Bus Stop:  Dhoby Ghaut Stn
No. 4 , Score: 0.00159 , Bus Stop:  Bedok Int
No. 5 , Score: 0.00139 , Bus Stop:  W'Lands Temp Int
OutDegree
No. 1 , Score: 0.00258 , Bus Stop:  Opp MAS Bldg
No. 2 , Score: 0.00218 , Bus Stop:  Boon Lay Int
No. 3 , Score: 0.00159 , Bus Stop:  The Esplanade
No. 4 , Score: 0.00159 , Bus Stop:  Dhoby Ghaut Stn Exit B
No. 5 , Score: 0.00139 , Bus Stop:  Bedok Int
Closeness
No. 1 , Score: 0.07005 , Bus Stop:  Mapletree Anson
No. 2 , Score: 0.06816 , Bus Stop:  Aft Capital Twr
No. 3 , Score: 0.06383 , Bus Stop:  Raffles Pl Stn Exit F
No. 4 , Score: 0.06382 , Bus Stop:  OPP SO

**2.2 b) Discuss the results and your observations (5 Points).** Based on the definitions and intuitions behind this 5 different centrality measures, discuss the results of 2.2 a):

* For each centrality, briefly describe what it means for a bus stop to have the highest score
* Compare the runtimes of the centrality measures and give reasons for the differences

1. What it means for a bus stop to have the highest score:
* PageRank: it has the most centrality that have overall most importance measured by quantity and quality of the other stops that link to it.
* InDegree: it has the largest number of incoming routes(edges), i.e., most routes stop at this stop
* OutDegree: it has the largest number of outgoing routes(edges), i.e., most routes leave this stop
* Closeness: its distance to all other stops is smallest, i.e., the sum of the shortest path distances from this stop to all other stops are smallest.
* Betweenness: it has most number of shortest paths between all other bus stops pass through this stop.

2. Runtimes(comment others and run seperately to get the runtime for each):
* InDegree(7.56ms) and OutDegree(3.57ms) are the fastest. Because they are just local measures and do not need extended information of the graph, so the calculation is easy and fast.
* PageRank(423ms) is the third fast. The calculation is more complex than local measures because it also needs to consider the extended information of the neighboring nodes in graph. It uses recursive method. But it's still scalable due to parallelization.
* Closeness(12.8s) and Betweenness(55.7s) are the slowest. They are both distance-based measures, and require to solve the All-Pairs Shortest Path Problem, and the calculation of shortest paths is non-trivial. The time complexity could be O(n^3).

### 2.3 Min-Cut Analysis

In the lecture, we talked about the Min-Cut problem and its application to find communities. While the notion of communities is not obvious for a bus network, problems due to interrupted connections a very relevant. For example, if the road between 2 bus stops is suddenly blocked due to a traffic accident, all bus routes going through this connections (i.e., edge in the graph) will be disrupted.

In this task, we use the simplified version of the Min-Cut problem the finds the Min-Cut with respect to a given source and target nodes (compared to the whole graph).

**2.3 a) Find the most "reliable" destinations from "Boon Lay Int" (bus stop code: 22009) (4 Points)**! A destination is reliable of it takes a lot of interrupted connections between two bus stops before the destination is no longer reachable from "Boon Lay Int". 

(Hint: Use [`nx.minimum_edge_cut()`](https://networkx.org/documentation/networkx-1.9/reference/generated/networkx.algorithms.connectivity.cuts.minimum_edge_cut.html) for convenience).

In [ ]:
origin = '22009' # Boon Lay Int
max_min_cut = -1
most_reliably_destinations = []


#########################################################################################
### Your code starts here ###############################################################
G = nx.DiGraph(A_mrt)
source = [k for k,v in node_map.items() if v == origin][0]

targets = list(G.nodes)
targets.remove(source)#remove itself
for node in targets:
    min_cut = nx.minimum_edge_cut(G, source, node)
    
    if max_min_cut < len(min_cut):
        most_reliably_destinations = [] #ensure only most reliable destinations in list
        if node_map[node] in bus_stop_map:
            most_reliably_destinations .append(bus_stop_map[node_map[node]])
        else:
            most_reliably_destinations .append(node_map[node])
        #print(nx.edge_connectivity(G, source, node))
        max_min_cut = len(min_cut)
    elif max_min_cut == len(min_cut):
        if node_map[node] in bus_stop_map:
            most_reliably_destinations .append(bus_stop_map[node_map[node]])  
        else:
            most_reliably_destinations .append(node_map[node])

        
### Your code ends here #################################################################
#########################################################################################          


print('The maximum Min-Cut between "Boon Lay Int" and any other bus stop is {}'.format(max_min_cut))
print('The most reliable destinations are {}'.format(most_reliably_destinations))

The maximum Min-Cut between "Boon Lay Int" and any other bus stop is 7
The most reliable destinations are ['Dhoby Ghaut Stn', 'Bedok Int', "W'Lands Temp Int", 'Mapletree Anson']


**2.3 b) Discussion of results (2 Points)**! Compare the result for `most_reliably_destinations` from 2.3 a) with the table in 2.2 a)! What observation can you make and how can you explain it?

'Boon Lay Int' is in the Top 5 bus stop lists (except for Closeness and Betweenness) with high centrality scores, and its most reliably destinations are also in the 5 centrality measures top lists(in 2.2a).

It might because that since 'Boon Lay Int' is clearly an important and central bus stop, in order for a stop to be consider reliable to 'Boon Lay Int' and have the most min-cut, there should be many connections between this stop and 'Boon Lay Int', which means that the reliable destination is also very connective and might have a high indegree/outdegree, etc. It is very likely that the reliable destinations also take a center position in bus map. Since the most reliable destinations are connected to the important stop('Boon Lay Int') with a very high centrality score, they will have a high score under these centrality measures for their connectivity, centrality, and importance in map and thus rank top on the lists.